In [1]:
import schedule_Generator as sg
import pandas as pd

Example
- Line 25: 25
- Stop ULB: 5407F
- Direction BOONDAEL GARE
- Date: SEP 07
- Time: 5-6 and after 20

In [123]:
trips = pd.read_csv('../data/gtfs3Sept/trips.csv')
trips_line = trips.loc[trips['route_id']==11,:]
trip_line_head = trips_line.loc[trips_line['trip_headsign']=='ROODEBEEK',:]
trip_line_head.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
93416,11,236487051,113017948236487051,ROODEBEEK,0,8925284,008t0013
93417,11,236487051,113017949236487051,ROODEBEEK,0,8925280,008t0013
93418,11,236487051,113017983236487051,ROODEBEEK,0,8925280,008t0005
93419,11,236487051,113017984236487051,ROODEBEEK,0,8925286,008t0011
93421,11,236487051,113018011236487051,ROODEBEEK,0,8925282,008t0011


In [124]:
calendar = pd.read_csv('../data/gtfs3Sept/calendar.csv')
calendar_week = calendar.loc[((calendar['start_date']<=20210907) & (calendar['end_date']>=20210907)),:]
calendar_week_day = calendar_week.loc[calendar_week['tuesday']==1,:]
calendar_week_day.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
3,236270000,1,1,1,1,1,0,0,20210901,20210917
15,237645001,1,1,1,1,1,0,0,20210906,20210917
26,237648001,1,1,1,1,1,0,0,20210906,20210917
34,237534000,1,1,1,1,1,0,0,20210901,20210917
43,236695000,1,1,1,1,1,0,0,20210901,20210917


In [125]:
trip_line_date_head = pd.merge(left=trip_line_head, right=calendar_week_day, on='service_id', how='inner').loc[:,['route_id','service_id','trip_id']]
trip_line_date_head.head()

,route_id,service_id,trip_id
0,11,236695000,113077506236695000
1,11,236695000,113077509236695000
2,11,236695000,113077516236695000
3,11,236695000,113077517236695000
4,11,236695000,113077519236695000


In [126]:
stop_times = pd.read_csv('../data/gtfs3Sept/stop_times.csv')
time_line_date_head = pd.merge(left=trip_line_date_head, right=stop_times, on='trip_id')
time_line_date_head_stop = time_line_date_head.loc[time_line_date_head['stop_id']=="1047F",:]
time_line_date_head_stop.sort_values('arrival_time')
time_line_date_head_stop_nbusy = time_line_date_head_stop.loc[((time_line_date_head_stop['arrival_time']<='06:00:00') & (time_line_date_head_stop['arrival_time']>='05:00:00')) | (time_line_date_head_stop['arrival_time']>='20:00:00'), :]
time_line_date_head_stop_nbusy.sort_values('arrival_time').count()

route_id          19
service_id        19
trip_id           19
arrival_time      19
departure_time    19
stop_id           19
stop_sequence     19
pickup_type       19
drop_off_type     19
dtype: int64

In [127]:
def unix_to_datetime(unix_time):
    return pd.to_datetime(unix_time,unit="ms",origin="unix")

def datetime_to_unix(dt):
    return int(pd.Timestamp.timestamp(pd.Timestamp(dt))*1000)

unix_to_datetime(1630998894287)

Timestamp('2021-09-07 07:14:54.287000')

In [128]:
actural_time = pd.concat([pd.read_csv('../data/vehiclePosition01.csv'),pd.read_csv('../data/vehiclePosition02.csv'),pd.read_csv('../data/vehiclePosition03.csv'),pd.read_csv('../data/vehiclePosition04.csv')])
actural_time_line = actural_time.loc[actural_time['LineID']==8,:]
actural_time_line_point = actural_time_line.loc[actural_time_line['PointID']==1047,:]
actural_time_line_point['Time'] = actural_time_line_point['Time'].apply(unix_to_datetime)
actural_time_line_point_date = actural_time_line_point.loc[actural_time_line_point['Time'].dt.date == pd.to_datetime('2021-09-07').date(),:]
actural_time_line_point_date['Time'] = (actural_time_line_point_date['Time'] + pd.Timedelta('02:00:00'))
actural_time_line_point_date_arrive = actural_time_line_point_date.loc[actural_time_line_point_date['DistanceFromPoint']<=200,:]
select_list = [True]
for i in range(1, len(actural_time_line_point_date_arrive)):
    if actural_time_line_point_date_arrive.iloc[i,0] - actural_time_line_point_date_arrive.iloc[i-1,0] <= pd.Timedelta('00:00:45'):
        select_list.append(False)
    else:
        select_list.append(True)
actural_time_line_point_date_arrive_noduplicate = actural_time_line_point_date_arrive.loc[select_list,:]
actural_time_line_point_date_arrive_noduplicate_nbusy = actural_time_line_point_date_arrive_noduplicate.loc[(actural_time_line_point_date_arrive_noduplicate['Time'] <= pd.to_datetime('2021-09-07 06:00:00')) & (actural_time_line_point_date_arrive_noduplicate['Time'] >= pd.to_datetime('2021-09-07 05:00:00')) | (actural_time_line_point_date_arrive_noduplicate['Time'] >= pd.to_datetime('2021-09-07 20:00:00')),:]
actural_time_line_point_date_arrive_noduplicate_nbusy.sort_values('Time').count()

/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_66825/2399112832.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actural_time_line_point['Time'] = actural_time_line_point['Time'].apply(unix_to_datetime)
/var/folders/v1/m2nqhx3d4yv9mjbpndrlsck00000gn/T/ipykernel_66825/2399112832.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actural_time_line_point_date['Time'] = (actural_time_line_point_date['Time'] + pd.Timedelta('02:00:00'))


Time                 17
LineID               17
DirectionID          17
DistanceFromPoint    17
PointID              17
dtype: int64

Schedule train 146
Actual train 62

In [129]:
on_time = 0
time_line_date_head_stop_nbusy['arrival_time'].apply(pd.to_timedelta)
actural_time_line_point_date_arrive_noduplicate_nbusy['Time'] - pd.Timestamp('2021-09-07 00:00:00')
for t in time_line_date_head_stop_nbusy['arrival_time'].apply(pd.to_timedelta):
    ot = False
    for at in actural_time_line_point_date_arrive_noduplicate_nbusy['Time'] - pd.Timestamp('2021-09-07 00:00:00'):
        t_dif = t - at
        if t_dif <= pd.Timedelta('00:00:45') and t_dif >= pd.Timedelta('-00:00:45'):
            ot = True
    if ot == True:
        on_time+=1
on_time_rate = on_time/len(time_line_date_head_stop_nbusy['arrival_time'])
on_time_rate

0.21052631578947367

In [130]:
actural_time_line_point_date_arrive.sort_values('Time')

,Time,LineID,DirectionID,DistanceFromPoint,PointID
1066927,2021-09-07 05:42:30.614,8,5229,6,1047
1067197,2021-09-07 05:43:02.974,8,5229,3,1047
1067454,2021-09-07 05:43:35.128,8,5229,132,1047
1087642,2021-09-07 06:13:40.483,8,5229,3,1047
1088085,2021-09-07 06:14:12.637,8,5229,6,1047
1088521,2021-09-07 06:14:44.075,8,5229,37,1047
1103465,2021-09-07 06:30:23.614,8,5229,0,1047
1104024,2021-09-07 06:30:53.925,8,5229,186,1047
1121216,2021-09-07 06:46:04.691,8,5229,0,1047
1121858,2021-09-07 06:46:37.869,8,5229,42,1047
